In [35]:
import os
import pandas as pd
import numpy as np
import datetime as dt
import decimal


def total_excessive_delay(df_ted):
    df_ted['TED'] = round(df_ted['ED'] * df_ted['PK_HR'], 2)
    df_ted['TED'] = round(df_ted['TED']) # Need this line to fix excel rounding weirdness

    #Groupby for summation by TMC
    #ted_operations = ({'TED' : 'sum'})
    #df_ted = df_ted.groupby('tmc_code', as_index=False).agg(ted_operations)
    return df_ted
    
def peak_hr(df_pk):
    df_pk['PK_HR'] = round(df_pk['DirAADT_AUTO']/4)
    return df_pk
    
def excessive_delay(df_ed):
    df_ed['ED'] = round(df_ed['RSD']/3600, 3) # check this value hundredths of an hour
    df_ed['ED'] = np.where(df_ed['ED'] >= 0, df_ed['ED'], 0)
    return df_ed

def RSD(df_rsd):
    # returns travel time segment delay calculations.
    df_rsd['RSD'] = df_rsd['SD'] - df_rsd['2017_EDT']
    df_rsd['RSD'] = np.where(df_rsd['RSD'] >= 0, df_rsd['RSD'], 0)
    return df_rsd
        
def segment_delay(df_sd):
    # returns SD spreadsheet value.
    df_sd['SD'] = df_sd['travel_time_seconds'].round()
    return df_sd

def AADT_splits(df_in):
    # apply ODOT methodology AADT_spit calculations to dataframe and create new columns.
    # df_in['aadt_singl] = Dir_AADT_BUD, 
    # df_in['aadt_combi'] = DirAADT_TRK
    df_in['DirAADT_AUTO'] = df_in['aadt'] - (df_in['aadt_singl'] + df_in['aadt_combi'])
    df_in['pct_auto'] = df_in['DirAADT_AUTO']/df_in['aadt']
    df_in['pct_bus'] = df_in['aadt_singl']/df_in['aadt']
    df_in['pct_truck'] = df_in['aadt_combi']/df_in['aadt']
    return df_in

def main():
    pd.set_option('display.max_rows', None)

    df = pd.read_csv(os.path.join(os.path.dirname('__file__'), 'Feb2017_test/Feb2017_test.csv')) #fix in script implementation
    # Filter by timestamps
    df['measurement_tstamp'] = pd.to_datetime(df['measurement_tstamp'])
    df = df[df['measurement_tstamp'].dt.weekday.isin([0, 1, 2, 3, 4])] # Capture weekdays only
    df = df[df['measurement_tstamp'].dt.hour.isin([6, 7, 8, 9, 16, 17, 18, 19])]
    
    df_meta = pd.read_csv(os.path.join(os.path.dirname('__file__'), 'Feb2017_test/TMC_Identification.csv'), 
                          usecols=['tmc', 'miles', 'tmclinear', 'aadt', 'aadt_singl', 'aadt_combi' ])
    df_odot = pd.read_csv(os.path.join(os.path.dirname('__file__'), 'Feb2017_test/odot_edt.csv'))    
    df = pd.merge(df, df_meta, left_on=df['tmc_code'], right_on=['tmc'], how='inner')
    df = pd.merge(df, df_odot, left_on=df['tmc_code'], right_on=['TMC'], how='inner')
    
    df = AADT_splits(df)
    df = segment_delay(df)
    df = RSD(df)
    df = excessive_delay(df)
    df = peak_hr(df)
    df = total_excessive_delay(df)
    #print(df)                                       
    # Return a selected row for test purposes only
    df = df[['RSD','PK_HR','ED', 'tmc_code', 'TED']]
    print(df.loc[df['tmc_code'] == '114-04368'])

In [36]:
main()

          RSD   PK_HR     ED   tmc_code    TED
192000    0.0  7107.0  0.000  114-04368    0.0
192001    2.0  7107.0  0.001  114-04368    7.0
192002    0.0  7107.0  0.000  114-04368    0.0
192003   29.0  7107.0  0.008  114-04368   57.0
192004    8.0  7107.0  0.002  114-04368   14.0
192005   26.0  7107.0  0.007  114-04368   50.0
192006    2.0  7107.0  0.001  114-04368    7.0
192007   10.0  7107.0  0.003  114-04368   21.0
192008    0.0  7107.0  0.000  114-04368    0.0
192009    7.0  7107.0  0.002  114-04368   14.0
192010   22.0  7107.0  0.006  114-04368   43.0
192011   31.0  7107.0  0.009  114-04368   64.0
192012    2.0  7107.0  0.001  114-04368    7.0
192013   46.0  7107.0  0.013  114-04368   92.0
192014   50.0  7107.0  0.014  114-04368  100.0
192015   15.0  7107.0  0.004  114-04368   28.0
192016   33.0  7107.0  0.009  114-04368   64.0
192017   48.0  7107.0  0.013  114-04368   92.0
192018   38.0  7107.0  0.011  114-04368   78.0
192019   43.0  7107.0  0.012  114-04368   85.0
192020   53.0